In [18]:
import graphlab

In [19]:
products = graphlab.SFrame('amazon_baby.gl/')

In [20]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [21]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

# 1.

In [22]:
def counter(dict, word):
    if word in dict:
        return dict[word]
    else:
        return 0

In [23]:
def whole_counter():
    for word in selected_words:
        products[word] = products['word_count'].apply(lambda x:counter(x,word))

In [24]:
whole_counter()

In [25]:
products['awesome'].sort().tail()

dtype: int
Rows: 10
[2, 2, 2, 3, 3, 3, 3, 3, 3, 4]

In [26]:
sums = {}
for word in selected_words:
    sums[word] = products[word].sum()

In [27]:
sums

{'amazing': 1363,
 'awesome': 2090,
 'awful': 383,
 'bad': 3724,
 'fantastic': 932,
 'great': 45206,
 'hate': 1220,
 'horrible': 734,
 'love': 42065,
 'terrible': 748,
 'wow': 144}

# 2.

In [28]:
train_data,test_data = products.random_split(.8, seed=0)

In [29]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [30]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [31]:
products

name,review,rating,word_count,awesome,great,fantastic
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",0,0,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",0,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'cute': 1,'help': 2, 'doll': 1, ...",0,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'be': 1,'is': 1, 'it': 1, 'as': ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1, 'and': 2,'all': 1, 'right': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'all': 1,'standarad': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, ""daughter's"": ...",0,0,0


In [32]:
selected_words_model = graphlab.logistic_classifier.create(train_data,target='sentiment',features=selected_words,validation_set=test_data)

ToolkitError: The following columns were expected but are missing: ['sentiment']

In [ ]:
selected_words_model['coefficients'].print_rows(12)

In [ ]:
print selected_words_model['coefficients']['value'].sort()

# 3.

In [ ]:
selected_words_model.evaluate(test_data)

In [ ]:
graphlab.canvas.set_target('ipynb')

In [ ]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [ ]:
giraffe_reviews['predicted_sentiment'] = selected_words_model.predict(giraffe_reviews, output_type='probability')

In [ ]:
giraffe_reviews.sort('predicted_sentiment').tail()

# 4.

In [ ]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [ ]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

In [ ]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [ ]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment',ascending=False).head()

In [ ]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

In [ ]:
print diaper_champ_reviews[0]['review']

In [ ]:
sentiment_model.evaluate(test_data)